# Description

This notebook is written to visualize an example set of bias adjustment results computed in this project's [BiasAdjust_model_run.ipynb](./BiasAdjust_model_run.ipynb) notebook. We examine the magnitude of the bias adjustment and it's geospatial and probability space structure.

# Setup

In [1]:
# import the packages we need to run this code
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utilities as util
import pickle

# import what we need for the statistical/visual comparison
from statsmodels.distributions.empirical_distribution import ECDF
import cartopy.crs as ccrs

As an example we focus on GFDL-CM3 and July ('mon07'). 

The example location we choose is Phoenix, AZ, USA.

In [2]:
# define the example month and model we will be studying
mname='GFDL-CM3'
eg_month=7

# define the location to look at the distributions
eg_loc=

# Load example bias-adjustment dictionary

In [3]:
# define the directory where the example analysis dictionary is located
eg_loadpath='./data/cmip5/ba_comparison/'

# define the location of the bias-adjustment dictionary output from the sister notebook
loadfile='egdata_BA_'+mname+'_mon{:02d}.pkl'.format(eg_month)

# load in the result
with open(eg_loadpath+loadfile, "rb") as input_file:
    eg_storage = pickle.load(input_file)

# Bias-adjustment comparison

### Setup

In [4]:
print(eg_storage.keys())

dict_keys(['month', 'model', 'forced_xsimadj_BA', 'forced_xsimadj_raw', 'xsimcal', 'xobscal', 'natural_xsimadj_BA', 'natural_xsimadj_raw'])


In [5]:
# import what we need for the statistical/visual comparison
from statsmodels.distributions.empirical_distribution import ECDF
import pandas as pd

## Crossing years for Forced Distribution

### Load Crossing Year Data

In [6]:
# define the loadpath for the crossing years of a particular threshold value
deltaC=1.07
cross_loadpath='./data/analysis/CMIP5_crossing_year_'+str(np.round(deltaC,2))+'C.csv'
# load the file from the save file for each model, reindex it with model names
cross_dat=pd.read_csv(cross_loadpath,index_col=[0])

# print the results for the example model
print(cross_dat.loc[mname])

CrossYear_1.07C    2013
FirstYear          1998
LastYear           2028
Name: GFDL-CM3, dtype: int64


### Get Forced Slice over the Crossing Years

In [15]:
# define the slice between the first and last year for this model
eg_crossing_slice=util.dt64_yrslice(
    cross_dat.loc[mname]['FirstYear'], cross_dat.loc[mname]['LastYear']
)
# slice the datasets accordingly
eg_storage['forced_xsimadj_BA']=eg_storage['forced_xsimadj_BA'].sel(time=eg_crossing_slice)
eg_storage['forced_xsimadj_raw']=eg_storage['forced_xsimadj_raw'].sel(time=eg_crossing_slice)


### Calculate Bias-adjustment magnitudes

In [16]:
# calculate the specific bias adjustment magnitude by differencing raw and BA
BA_magnitude_forced=eg_storage['forced_xsimadj_BA']-eg_storage['forced_xsimadj_raw']
BA_magnitude_natural=eg_storage['natural_xsimadj_BA']-eg_storage['natural_xsimadj_raw']

## Map Example bias adjustments at the Median ($q_i$=0.5)

In [45]:
# wrap around the prime meridian before plotting
BA_forced_cyc = util.xr_add_cyclic_point(BA_magnitude_forced.TMAX, 'lon')
BA_natural_cyc = util.xr_add_cyclic_point(BA_magnitude_natural.TMAX, 'lon')

In [46]:
# define the quantile we will analyze over time
qi=0.5

In [ ]:
# setup things for the plot
levs=np.arange(-12,12+1,1)

# plot the result
fig=plt.figure(figsize=(12,12))

## FORCED
# create a subplot
ax1 = plt.subplot(2, 1, 1, projection=ccrs.PlateCarree())
# plot the 
datp1=ax1.contourf(BA_forced_cyc.lon,BA_forced_cyc.lat,
                   BA_forced_cyc.quantile(qi,dim='time'), \
                        levels=levs, transform=ccrs.PlateCarree(), \
                        cmap='RdBu_r', extend='both')
# draw the coastlines
ax1.coastlines()
ax1.gridlines(alpha=0.25,linestyle=(0, (5, 1)) )
# fig.colorbar(datp1)
# add the scenario string
plt.text(-165,-53,'Forced',color='black',fontsize=22,transform=ccrs.PlateCarree())
plt.title(mname+', absolute value of $T_{max}$ bias adjustments ($^{\circ}$C)',fontsize=22)

## NATURAL
# create a subplot
ax2 = plt.subplot(2, 1, 2, projection=ccrs.PlateCarree())
# plot the 
datp2=ax2.contourf(BA_natural_cyc.lon,BA_natural_cyc.lat,
                   BA_natural_cyc.quantile(qi,dim='time'), \
                        levels=levs, transform=ccrs.PlateCarree(), \
                        cmap='RdBu_r', extend='both')
# draw the coastlines
ax2.coastlines()
ax2.gridlines(alpha=0.25,linestyle=(0, (5, 1)) )
# add the scenario string
plt.text(-165,-53,'Natural',color='black',fontsize=22,transform=ccrs.PlateCarree())

# make a single large colorbar
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.25, 0.025, 0.5])
ticklabs = cbar_ax.get_yticklabels()
cbar_ax.set_yticklabels(levs, fontsize=14)
fig.colorbar(datp1, cax=cbar_ax)

# adjust the white space between the subplots
plt.subplots_adjust(wspace=0.05, hspace=0.)
plt.show()

Note that the bias adjustments between the two scenarios have quite a similar magnitude at the median.